<a href="https://colab.research.google.com/github/Sujal3141/Sujal/blob/main/House%20price%20prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install kaggle

In [7]:
!mkdir ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle/’: File exists


In [8]:
!cp kaggle.json ~/.kaggle/kaggle.json

In [9]:
!kaggle competitions download -c house-prices-advanced-regression-techniques

house-prices-advanced-regression-techniques.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
!unzip house-prices-advanced-regression-techniques

Archive:  house-prices-advanced-regression-techniques.zip
replace data_description.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: data_description.txt    
replace sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: sample_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: train.csv               


In [11]:
import pandas as pd
import matplotlib as plt
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier

In [25]:
test = pd.read_csv('test.csv')
train = pd.read_csv('train.csv')
x_train = train.drop('SalePrice',axis=1)
y_train = train.iloc[:,-1:]

In [13]:
train.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [29]:
#simple imputer
trf1 = ColumnTransformer([('imputed_LotFrontage',SimpleImputer(),[3]),
                          ('imputed_Alley',SimpleImputer(strategy = 'most_frequent'),[6]),
                          ('imputed_MasVnrType',SimpleImputer(strategy = 'most_frequent'),[25]),
                          ('imputed_MasVnrArea',SimpleImputer(),[26]),
                          ('imputed_BsmtQual',SimpleImputer(strategy='most_frequent'),[30]),
                          ('imputed_BsmtCond',SimpleImputer(strategy='most_frequent'),[31]),
                          ('imputed_BsmtExposure',SimpleImputer(strategy='most_frequent'),[32]),
                          ('imputed_BsmtFinType1',SimpleImputer(strategy='most_frequent'),[33]),
                          ('imputed_BsmtFinType2',SimpleImputer(strategy='most_frequent'),[35]),
                          ('imputed_Electrical',SimpleImputer(strategy='most_frequent'),[42]),
                          ('imputed_FireplaceQu',SimpleImputer(strategy='most_frequent'),[57]),
                          ('imputed_GarageType',SimpleImputer(strategy='most_frequent'),[60]),
                          ('imputed_GarageQual',SimpleImputer(strategy='most_frequent'),[63]),
                          ('imputed_GarageCond',SimpleImputer(strategy='most_frequent'),[64]),
                          ('imputed_PoolQC',SimpleImputer(strategy='most_frequent'),[72]),
                          ('imputed_Fence',SimpleImputer(strategy='most_frequent'),[73]),
                          ('imputed_MiscFeature',SimpleImputer(strategy='most_frequent'),[74]),


                      ],remainder = 'passthrough')

In [84]:
#Making a list of columns which are of datatypes objects
object_dt = train.select_dtypes(include='object')

['MSZoning',
 'Street',
 'Alley',
 'LotShape',
 'LandContour',
 'Utilities',
 'LotConfig',
 'LandSlope',
 'Neighborhood',
 'Condition1',
 'Condition2',
 'BldgType',
 'HouseStyle',
 'RoofStyle',
 'RoofMatl',
 'Exterior1st',
 'Exterior2nd',
 'MasVnrType',
 'ExterQual',
 'ExterCond',
 'Foundation',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Heating',
 'HeatingQC',
 'CentralAir',
 'Electrical',
 'KitchenQual',
 'Functional',
 'FireplaceQu',
 'GarageType',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PavedDrive',
 'PoolQC',
 'Fence',
 'MiscFeature',
 'SaleType',
 'SaleCondition']

In [82]:
list_object_dt = list(object_dt)
k= list_object_dt
for i in range(len(list_object_dt)):
  list_object_dt[i]=train.columns.get_loc(list_object_dt[i])


In [89]:
#vARIABLE ENCODING
trf2 = ColumnTransformer([('encoding_MSZoning',OneHotEncoder(),[2]),
('encoding_Street',OneHotEncoder(),[5]),
('encoding_Alley',OneHotEncoder(),[6]),
('encoding_LotShape',OneHotEncoder(),[7]),
('encoding_LandContour',OneHotEncoder(),[8]),
('encoding_Utilities',OneHotEncoder(),[9]),
('encoding_LotConfig',OneHotEncoder(),[10]),
('encoding_LandSlope',OneHotEncoder(),[11]),
('encoding_Neighborhood',OneHotEncoder(),[12]),
('encoding_Condition1',OneHotEncoder(),[13]),
('encoding_Condition2',OneHotEncoder(),[14]),
('encoding_BldgType',OneHotEncoder(),[15]),
('encoding_HouseStyle',OneHotEncoder(),[16]),
('encoding_RoofStyle',OneHotEncoder(),[21]),
('encoding_RoofMatl',OneHotEncoder(),[22]),
('encoding_Exterior1st',OneHotEncoder(),[23]),
('encoding_Exterior2nd',OneHotEncoder(),[24]),
('encoding_MasVnrType',OneHotEncoder(),[25]),
('encoding_ExterQual',OneHotEncoder(),[27]),
('encoding_ExterCond',OneHotEncoder(),[28]),
('encoding_Foundation',OneHotEncoder(),[29]),
('encoding_BsmtQual',OneHotEncoder(),[30]),
('encoding_BsmtCond',OneHotEncoder(),[31]),
('encoding_BsmtExposure',OneHotEncoder(),[32]),
('encoding_BsmtFinType1',OneHotEncoder(),[33]),
('encoding_BsmtFinType2',OneHotEncoder(),[35]),
('encoding_Heating',OneHotEncoder(),[39]),
('encoding_HeatingQC',OneHotEncoder(),[40]),
('encoding_CentralAir',OneHotEncoder(),[41]),
('encoding_Electrical',OneHotEncoder(),[42]),
('encoding_KitchenQual',OneHotEncoder(),[53]),
('encoding_Functional',OneHotEncoder(),[55]),
('encoding_FireplaceQu',OneHotEncoder(),[57]),
('encoding_GarageType',OneHotEncoder(),[58]),
('encoding_GarageFinish',OneHotEncoder(),[60]),
('encoding_GarageQual',OneHotEncoder(),[63]),
('encoding_GarageCond',OneHotEncoder(),[64]),
('encoding_PavedDrive',OneHotEncoder(),[65]),
('encoding_PoolQC',OneHotEncoder(),[72]),
('encoding_Fence',OneHotEncoder(),[73]),
('encoding_MiscFeature',OneHotEncoder(),[74]),
('encoding_SaleType',OneHotEncoder(),[78]),
('encoding_SaleCondition',OneHotEncoder(),[79])],remainder='passthrough')










In [90]:
trf3 = DecisionTreeClassifier()

In [91]:
pipe = make_pipeline(trf1,trf2,trf3)


In [92]:
pipe.fit(x_train,y_train)

ValueError: For a sparse output, all columns should be a numeric or convertible to a numeric.

In [88]:

for i in range(len(list_object_dt)) :
  print(f"('encoding_{list(object_dt)[i]}',OneHotEncoder(),[{list_object_dt[i]}]),")

('encoding_MSZoning',OneHotEncoder(),[2]),
('encoding_Street',OneHotEncoder(),[5]),
('encoding_Alley',OneHotEncoder(),[6]),
('encoding_LotShape',OneHotEncoder(),[7]),
('encoding_LandContour',OneHotEncoder(),[8]),
('encoding_Utilities',OneHotEncoder(),[9]),
('encoding_LotConfig',OneHotEncoder(),[10]),
('encoding_LandSlope',OneHotEncoder(),[11]),
('encoding_Neighborhood',OneHotEncoder(),[12]),
('encoding_Condition1',OneHotEncoder(),[13]),
('encoding_Condition2',OneHotEncoder(),[14]),
('encoding_BldgType',OneHotEncoder(),[15]),
('encoding_HouseStyle',OneHotEncoder(),[16]),
('encoding_RoofStyle',OneHotEncoder(),[21]),
('encoding_RoofMatl',OneHotEncoder(),[22]),
('encoding_Exterior1st',OneHotEncoder(),[23]),
('encoding_Exterior2nd',OneHotEncoder(),[24]),
('encoding_MasVnrType',OneHotEncoder(),[25]),
('encoding_ExterQual',OneHotEncoder(),[27]),
('encoding_ExterCond',OneHotEncoder(),[28]),
('encoding_Foundation',OneHotEncoder(),[29]),
('encoding_BsmtQual',OneHotEncoder(),[30]),
('encoding_Bsm